<a href="https://colab.research.google.com/github/anirban1221/Learning_PyTorch/blob/main/RNN_application_Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
df=pd.read_csv('/content/100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [17]:
## tokenize the dataset
def tokenize(text):
  test=text.lower()
  text=text.replace('?','')
  text=text.replace("'","")
  return text.split()

## forming the vocabulary
vocab={'<UNK>':0}
def build_vocab(row):
  tokenized_question=tokenize(row['question'])
  tokenized_answer=tokenize(row['answer'])
  merged_tokens=tokenized_question+tokenized_answer

  for token in merged_tokens:
    if token not in vocab:
      vocab[token]=len(vocab)
## words to numerical indices
def text_to_indices(text,vocab):
  indexed_text=[]
  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [18]:
text_to_indices("are you fine?",vocab)

[0, 0, 0]

In [19]:
df.apply(build_vocab,axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [20]:
len(vocab)

330

In [21]:
import torch
from torch.utils.data import Dataset, DataLoader
class QADataset(Dataset):
  def __init__(self,df,vocab):
    self.df=df
    self.vocab=vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self,index):
    numerical_question=text_to_indices(self.df.iloc[index]['question'],self.vocab)
    numerical_answer=text_to_indices(self.df.iloc[index]['answer'],self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [22]:
dataset=QADataset(df,vocab)

In [23]:
dataloader=DataLoader(dataset,batch_size=1, shuffle=True)

In [24]:
for question,answer in dataloader:
  print(question,answer[0])


tensor([[ 42,  18, 120,   3, 191, 192]]) tensor([193])
tensor([[ 42,   2,   3, 215, 141, 173, 216, 174]]) tensor([115])
tensor([[ 42,  18,   2,   3, 287, 147,   3, 288]]) tensor([210])
tensor([[42, 43, 44, 45, 46, 47, 48]]) tensor([49])
tensor([[ 42, 141,   2, 231, 147,   3, 232, 233]]) tensor([160])
tensor([[ 10, 314,   3, 315, 316]]) tensor([317])
tensor([[  1,   2,   3, 151,  88,  19, 197, 198]]) tensor([199])
tensor([[ 42, 268, 269,  14, 270, 271, 163, 272]]) tensor([273])
tensor([[ 1,  2,  3, 24, 25,  5, 26, 19, 27]]) tensor([28])
tensor([[ 42, 318,   2, 319,  63,  64,   3, 320, 321]]) tensor([322])
tensor([[  1,  89, 234, 235, 236, 237]]) tensor([238])
tensor([[  1,   2,   3,   4,   5, 139]]) tensor([140])
tensor([[  1,   2,   3, 168, 169, 170,  85,  86]]) tensor([171])
tensor([[ 42, 221, 120, 222, 223,  19,  14, 224,  43]]) tensor([225])
tensor([[ 42, 324,   2,  63,  64,   3, 325,   5, 326]]) tensor([327])
tensor([[ 10, 144,   3, 145, 275, 276, 277,   5,   3, 278]]) tensor([279]

In [25]:
## constructing the RNN:
import torch.nn as nn

class SimpleRNN(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.embedding=nn.Embedding(vocab_size,embedding_dim=50)
    self.rnn=nn.RNN(50,64, batch_first=True)
    self.fc=nn.Linear(64,vocab_size)

  def forward(self,question):
    embedded_qn=self.embedding(question)
    hidden,final=self.rnn(embedded_qn)
    output=self.fc(final.squeeze(0))

    return output


In [26]:
learning_rate=0.001
epochs=20

In [27]:

model=SimpleRNN(len(vocab))

criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [29]:
## training_loop

for epoch in range(epochs):
  total_loss=0

  for question,answer in dataloader:
    optimizer.zero_grad()

    ## forward pass
    output=model(question)
    loss=criterion(output,answer[0])
    loss.backward()
    optimizer.step()
    total_loss+=loss.item()

  print(f"Epoch: {epoch+1}, Loss:{total_loss:4f}")


Epoch: 1, Loss:524.989253
Epoch: 2, Loss:461.568373
Epoch: 3, Loss:385.126272
Epoch: 4, Loss:318.670826
Epoch: 5, Loss:265.112942
Epoch: 6, Loss:215.456574
Epoch: 7, Loss:170.159732
Epoch: 8, Loss:131.331144
Epoch: 9, Loss:99.419566
Epoch: 10, Loss:75.503796
Epoch: 11, Loss:57.603286
Epoch: 12, Loss:45.110510
Epoch: 13, Loss:36.065999
Epoch: 14, Loss:29.603865
Epoch: 15, Loss:24.224943
Epoch: 16, Loss:19.817546
Epoch: 17, Loss:16.831985
Epoch: 18, Loss:14.598554
Epoch: 19, Loss:12.529571
Epoch: 20, Loss:11.006018


In [36]:
def predict(model, question, threshold=0.5):
  ## convert question to numbers
  numerical_question=text_to_indices(question,vocab)

  question_tensor=torch.tensor(numerical_question).unsqueeze(0)
  output=model(question_tensor)

  ## convert logits to probabilities
  probs=torch.nn.functional.softmax(output,dim=1)

  ## find index of max prob
  value,index=torch.max(probs,dim=1)

  if value<threshold:
    print("I don't know" )
  else:
    print(list(vocab.keys())[index])

In [38]:
predict(model,"what is the largest planet of our solar system")

Jupiter


In [37]:
predict(model,'are you mad?')

I don't know
